In [1]:
import sqlalchemy as sql
import pandas as pd
import glob
from os import path, sep
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift
from scipy.signal import oaconvolve
from scipy.signal import remez
import time
import decimate as d
from importlib import reload
reload(d);

In [2]:
time.perf_counter()

7912.382542937

In [3]:
f3 = remez(4000,[0., 0.5, 1., 250.],[1.,0.],fs=500)


In [4]:
server = "thesis.ca6j6heoraog.eu-central-1.rds.amazonaws.com"
engine = sql.create_engine(f"mysql+pymysql://admin:FphvsYQek4@{server}/thesis_db")

In [5]:
con = engine.connect()

```SQL
create table interpolation_types (
	id int(11) NOT NULL PRIMARY KEY AUTO_INCREMENT,
    interp_algo varchar(45),
    algo_version varchar(20),
    notebook_name varchar(45),
    sampling_rate float(6,4)
    )
```

In [12]:
stmnt = """
SELECT DISTINCT series_type_id, group_id, participant_id
FROM raw_interval_data
"""

raw_series = con.execute(stmnt)

In [7]:
raw_series

In [9]:
get_series = sql.text("""
SELECT data_time
FROM raw_interval_data
WHERE series_type_id = :sti
AND group_id = :gi
AND participant_id = :pi""")

In [13]:
tref = time.perf_counter()
for series_type_id, group_id, participant_id in raw_series:
    if series_type_id != 4:
        continue
    print(series_type_id, group_id, participant_id, np.around(time.perf_counter()-tref,2) ) 
    q = con.execute(get_series, sti=series_type_id, gi= group_id, pi=participant_id)
    t = np.array(q.fetchall()).astype(float).squeeze()
    x = t[:-1] # don't use the point which marks the end of the experiment
    y = np.diff(t)
    x_pw = np.arange(0,t[-1],0.5)
    y_pw = d.pw_constant_convolve(x, y, t[-1], f3, 4, 3, x_pw)
    nx = len(x_pw)
    interpolated_data = pd.DataFrame({
        'series_type_id': [series_type_id]*nx,
        'interpolation_series_id': [3]*nx,
        'group_id': [group_id]*nx,
        'participant_id': [participant_id]*nx,
        'data_ordinal': range(nx),
        'data_time': x_pw,
        'data_value': np.around(y_pw,5)
    })
    interpolated_data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi')   


4 1001 101 0.0
4 1001 102 0.38
4 1001 103 0.74
4 1003 108 1.12
4 1003 109 1.44
4 1004 110 1.79
4 1004 111 2.15
4 1004 112 2.5
4 1005 113 2.86
4 1005 114 3.22
4 1005 115 3.53
4 1006 116 3.88
4 1006 117 4.24
4 1006 118 4.6
4 1007 119 5.02
4 1007 120 5.39
4 1007 121 5.76
4 1008 122 6.11
4 1008 123 6.44
4 1008 124 6.8
4 1009 125 7.17
4 1009 126 7.53
4 1009 127 7.89
4 1010 128 8.29
4 1010 129 8.62
4 1010 130 8.96
4 1011 131 9.31
4 1011 132 9.64
4 1011 133 9.99
4 1012 134 10.33
4 1012 135 10.66
4 1012 136 11.01
4 1013 137 11.35
4 1013 138 11.69
4 1013 139 12.07
4 1014 140 12.43
4 1014 141 12.76
4 1014 142 13.12
4 1015 144 13.48
4 1015 145 13.82
4 1016 146 14.18
4 1016 147 14.55
4 1016 148 14.88
4 1017 149 15.26
4 1017 151 15.59
4 1019 155 15.92
4 1019 156 16.28
4 1019 157 16.62
4 1020 158 16.96
4 1020 159 17.32
4 1020 160 17.65
4 1022 164 17.98
4 1022 165 18.31
4 1022 166 18.72
4 1023 167 19.06
4 1023 169 19.43
4 1024 170 19.78
4 1024 171 20.14
4 1024 172 20.5
4 1025 173 20.86
4 1025 174 21.

In [ ]:
y_pw = d.pw_constant_convolve(x, y, t[-1], f3, 4, 3, x_pw)
nx = len(x_pw)
interpolated_data = pd.DataFrame({
    'series_type_id': [series_type_id]*nx,
    'group_id': [group_id]*nx,
    'participant_id': [participant_id]*nx,
    'data_ordinal': range(nx),
    'data_time': x_pw,
    'data_value': np.around(y_pw,5)
})
interpolated_data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi')   


In [ ]:
df

In [ ]:
(t[-1], f3, 4, 3, x_pw)

In [ ]:
x

In [ ]:
y